In [2]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

from dotenv import load_dotenv
load_dotenv()

jjkim_key = os.getenv("JINA_API_JJKIM")
obj76_key = os.getenv("JINA_API_OBJECTS76")

jjkim_key, obj76_key

('jina_de01014707b44f2b8da84917e486c09e_y-mapgpzKFuLA9iutoemgnK1eRw',
 'jina_7486fd840bda4ea9b428afbaab1d62706gksDo-ypWXbUO7QHwzukzeBujN_')

In [ ]:
import yaml
import subprocess

def get_template(template_path):
    template_yaml = Path(template_path).expanduser()
    if not template_yaml.exists():
        raise ValueError(f'not found: {template_yaml}')


In [ ]:

    config = yaml.safe_load(template_yaml.read_text(encoding='utf-8')) or {}
    templates = config.get('TEMPLATES', {})
    if not isinstance(templates, dict) or not templates:
        raise ValueError('no templates')

    # ~/.config/rofi/
    cmd = [
        'rofi', '-dmenu', '-no-custom',
        '-monitor', '-4', # 4:active window
        '-theme-str', 'window {width: 10%;} entry { enabled: false; }',
        '-p', 'Choose template:',
        '-dpi', '192',
        '-lines', f'{len(templates)}',
        '-no-fixed-num-lines',
    ]

    rofi = subprocess.run(
        cmd,
        input='\n'.join(templates.keys()),
        text=True,
        capture_output=True,
    )

    choice = (rofi.stdout or '').strip()
    if not choice or choice not in templates:
        return "", ""
    return choice, templates[choice]


get_template("~/.config/rofi/template.yaml")

In [ ]:

from dotenv import load_dotenv
load_dotenv(os.path.expanduser("~/.config/rofi/.env"))
jjkim_key = os.getenv("JINA_API_KEY_JJKIM")
obj76_key = os.getenv("JINA_API_KEY_OBJ76")
jjkim_key, obj76_key,